In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [5]:
train_df = pd.read_csv("/content/training.csv")
valid_df = pd.read_csv("/content/validation.csv")
test_df = pd.read_csv("/content/test.csv")

In [6]:
new_df = pd.concat([train_df , valid_df , test_df] , axis=0 )
# Reset the index
new_df = new_df.reset_index(drop=True)

In [23]:
new_df.head(10)

,text,label,one_hot_label
0,i didnt feel humiliated,0,"[1, 0, 0, 0, 0, 0]"
1,i can go from feeling so hopeless to so damned...,0,"[1, 0, 0, 0, 0, 0]"
2,im grabbing a minute to post i feel greedy wrong,3,"[0, 0, 0, 1, 0, 0]"
3,i am ever feeling nostalgic about the fireplac...,2,"[0, 0, 1, 0, 0, 0]"
4,i am feeling grouchy,3,"[0, 0, 0, 1, 0, 0]"
5,ive been feeling a little burdened lately wasn...,0,"[1, 0, 0, 0, 0, 0]"
6,ive been taking or milligrams or times recomme...,5,"[0, 0, 0, 0, 0, 1]"
7,i feel as confused about life as a teenager or...,4,"[0, 0, 0, 0, 1, 0]"
8,i have been with petronas for years i feel tha...,1,"[0, 1, 0, 0, 0, 0]"
9,i feel romantic too,2,"[0, 0, 1, 0, 0, 0]"


In [8]:
for i in range (6):

    print(new_df[new_df["label"] == i].shape)

(5797, 2)
(6761, 2)
(1641, 2)
(2709, 2)
(2373, 2)
(719, 2)


In [9]:
def make_one_hot(index):
    label = np.zeros(6).astype(int)
    # label = [0,0,0,0,0,0]
    label[index] = 1
    return label

In [10]:
make_one_hot(4)

array([0, 0, 0, 0, 1, 0])

In [11]:
new_df["one_hot_label"] = new_df["label"].apply(make_one_hot)

In [62]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_data = tokenizer(new_df["text"].values.tolist(), return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)

labels = np.stack(np.array(new_df["label"]))  # Label is already an array of 0 and 1

In [68]:
tokenized_data

{'input_ids': array([[  101,   178,  1238, ...,     0,     0,     0],
        [  101,   178,  1169, ...,     0,     0,     0],
        [  101, 13280, 10810, ...,     0,     0,     0],
        ...,
        [  101,   178,  1631, ...,     0,     0,     0],
        [  101, 13280,  2296, ...,     0,     0,     0],
        [  101,   178,  1631, ...,     0,     0,     0]]),
 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [80]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased" , num_labels = 6)
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))  # No loss argument!


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
type(model)

transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification

In [82]:
model.fit(tokenized_data, labels , epochs = 5)

Epoch 1/5
625/625 [==============================] - 392s 562ms/step - loss: 0.4288
Epoch 2/5
625/625 [==============================] - 348s 557ms/step - loss: 0.1275
Epoch 3/5
625/625 [==============================] - 348s 557ms/step - loss: 0.1008
Epoch 4/5
625/625 [==============================] - 348s 557ms/step - loss: 0.0937
Epoch 5/5
625/625 [==============================] - 348s 557ms/step - loss: 0.0911


In [103]:
tokenized_pred = tokenizer("im feeling so sad right now", return_tensors="np", padding=True)

pred = model.predict(tokenized_pred["input_ids"])

tf.nn.softmax(pred.logits, axis=1)

1/1 [==============================] - 0s 141ms/step


<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[9.9965668e-01, 5.5941153e-05, 1.1776631e-04, 5.8835139e-05,
        5.3218955e-05, 5.7636447e-05]], dtype=float32)>

In [87]:
tokenized_test = tokenizer(test_df["text"].values.tolist() , return_tensors="np", padding=True)

In [92]:
test_labels = test_df["label"].values

In [94]:
y_pred = model.predict(tokenized_test["input_ids"])

63/63 [==============================] - 10s 157ms/step


In [105]:
y_pred = tf.nn.softmax(y_pred.logits , axis = 1)

In [111]:
y_pred = (y_pred.numpy() > 0.5 ).astype(int)

In [116]:
# test_labels
y_true = []

for label in test_labels:
  one_hot = make_one_hot(label)
  y_true.append(one_hot)

In [119]:
y_true = np.stack(y_true)

In [120]:
from sklearn.metrics import classification_report

print(classification_report(y_true , y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       581
           1       0.96      0.94      0.95       695
           2       0.79      0.94      0.86       159
           3       0.97      0.92      0.94       275
           4       0.96      0.86      0.90       224
           5       0.72      0.88      0.79        66

   micro avg       0.94      0.94      0.94      2000
   macro avg       0.89      0.92      0.90      2000
weighted avg       0.94      0.94      0.94      2000
 samples avg       0.94      0.94      0.94      2000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
